__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2023-01-27 16:27:15 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2023-01-27 16:27:15 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2023-01-27 16:27:17 | AdsListParser: downloader init
[INFO    ] 2023-01-27 16:27:17 | AvitoParser: init
[INFO    ] 2023-01-27 16:27:22 | AvitoParser: 91 pages for read
[INFO    ] 2023-01-27 16:27:22 | AdsListParser: start read and parse pages...
[INFO    ] 2023-01-27 16:27:25 | AdsListParser: read page 1
[INFO    ] 2023-01-27 16:27:28 | AdsListParser: read page 2
[INFO    ] 2023-01-27 16:27:31 | AdsListParser: read page 3
[INFO    ] 2023-01-27 16:27:36 | AdsListParser: read page 4
[INFO    ] 2023-01-27 16:27:42 | AdsListParser: read page 5
[INFO    ] 2023-01-27 16:27:45 | AdsListParser: read page 6
[INFO    ] 2023-01-27 16:27:51 | AdsListParser: read page 7
[INFO    ] 2023-01-27 16:27:54 | AdsListParser: read page 8
[INFO    ] 2023-01-27 16:27:57 | AdsListParser: read page 9
[INFO    ] 2023-01-27 16:28:54 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

4115
3017


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
2947,2308561511,"Объявление «1-к. квартира, 60 м², 3/3 эт.» 6 фотографий",8721270,ЖК «Голубая Бухта»,,"ЖК «Голубая Бухта» — это неповторимое сочетание современной архитектуры и комфортной жизни с удобным расположением у самого берега Черного моря и живописным видом на Голубую бухту. \nКомплекс имеет уникальную инфраструктуру:300м до пляжа «Голубая бухта», Sp",89,2023-01-27 16:33:39.614491,sevastopol
1131,2265190654,"Объявление «3-к. квартира, 73,2 м², 8/9 эт.» 25 фотографий",9000000,,,"Трехкомнатная квартира 73,2 м2 на 8/9 этаже дома. Расположена в г. Севастополь, Ленинский район, проспект Генерала Острякова, д 202. \n\nЭта квартира идеально подойдет для постоянного проживания большой и дружной семьи. \n\nПозвоните нам, чтобы узнать все дета",26,2023-01-27 16:33:39.614491,sevastopol


In [11]:
# !pip install dash_table